# Exercise Detection Accuracy Report

This notebook tests how well our AI coach can tell if you're doing exercises correctly or incorrectly.

## What We're Testing

We recorded people doing 5 different exercises:
- Bicep curls
- Lateral raises
- Overhead press
- Front kicks
- Squats

For each exercise, some people did it correctly and some did it incorrectly on purpose. We then checked if our AI could tell the difference.

## How to Read the Results

- **Accuracy**: How often the AI is right overall
- **Precision**: When the AI says "correct form", how often is it actually correct?
- **Recall**: When someone does correct form, how often does the AI catch it?
- **Specificity**: When someone does incorrect form, how often does the AI catch it?
- **F1-Score**: A combined score that balances precision and recall


## Step 1: Load the Test Data

We start by loading our test data from a CSV file. This file contains:
- Which exercise was performed
- Whether it was done correctly or incorrectly (ground truth)
- What our AI predicted


In [1]:
import pandas as pd
import json

df = pd.read_csv('test.csv')
print(f"Total samples loaded: {len(df)}")
df.head()

Total samples loaded: 101


,exercise,ground_truth,prediction
0,bicep_curl,incorrect,incorrect
1,bicep_curl,incorrect,incorrect
2,bicep_curl,incorrect,incorrect
3,bicep_curl,incorrect,incorrect
4,bicep_curl,incorrect,incorrect


## Step 2: Clean the Data

Sometimes data has extra spaces or formatting issues. We clean it up here to make sure everything works properly.


In [2]:
# Remove trailing/leading spaces from column names
df.columns = df.columns.str.strip()

# Remove trailing/leading spaces from all string values
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Check cleaned data
print("Cleaned columns:", df.columns.tolist())
print("\nExercises found:", df['exercise'].unique())
print(f"\nTotal samples: {len(df)}")
df.head()

Cleaned columns: ['exercise', 'ground_truth', 'prediction']

Exercises found: ['bicep_curl' 'lateral_raise' 'overhead_press' 'front_kicks' 'squat']

Total samples: 101


,exercise,ground_truth,prediction
0,bicep_curl,incorrect,incorrect
1,bicep_curl,incorrect,incorrect
2,bicep_curl,incorrect,incorrect
3,bicep_curl,incorrect,incorrect
4,bicep_curl,incorrect,incorrect


## Step 3: Check Data Distribution

Let's see how many test samples we have for each exercise. This helps us make sure we're testing fairly across all exercises.


In [13]:
# check how records are distributed across exercises
exercise_counts = df['exercise'].value_counts()
print("\nRecords per exercise:")
print(exercise_counts)



Records per exercise:
exercise
overhead_press    21
bicep_curl        20
lateral_raise     20
front_kicks       20
squat             20
Name: count, dtype: int64


## Step 4: Define Our Calculation Function

This function calculates important metrics:

### Understanding the Confusion Matrix:
- **TP (True Positive)**: AI said "correct" and it WAS correct ✅
- **TN (True Negative)**: AI said "incorrect" and it WAS incorrect ✅
- **FP (False Positive)**: AI said "correct" but it was actually incorrect ❌
- **FN (False Negative)**: AI said "incorrect" but it was actually correct ❌

### The Metrics:
- **Accuracy** = (TP + TN) / Total → How often is the AI right?
- **Precision** = TP / (TP + FP) → When AI says "correct", how reliable is it?
- **Recall** = TP / (TP + FN) → Does the AI catch all the correct forms?
- **Specificity** = TN / (TN + FP) → Does the AI catch all the incorrect forms?
- **F1-Score** → A balanced score combining precision and recall


In [ ]:
def calculate_metrics(exercise_df):
    """Calculate confusion matrix and all metrics for one exercise"""
    TP = len(exercise_df[(exercise_df['ground_truth'] == 'correct') & 
                          (exercise_df['prediction'] == 'correct')])
    FP = len(exercise_df[(exercise_df['ground_truth'] == 'incorrect') & 
                          (exercise_df['prediction'] == 'correct')])
    TN = len(exercise_df[(exercise_df['ground_truth'] == 'incorrect') & 
                          (exercise_df['prediction'] == 'incorrect')])
    FN = len(exercise_df[(exercise_df['ground_truth'] == 'correct') & 
                          (exercise_df['prediction'] == 'incorrect')])
    
    total = TP + FP + TN + FN
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    fpr = FP / (FP + TN) if (FP + TN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / total if total > 0 else 0
    
    return {
        'TP': TP, 'FP': FP, 'TN': TN, 'FN': FN,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'fpr': fpr,
        'f1_score': f1,
        'accuracy': accuracy,
        'total': total
    }

## Step 5: Calculate Metrics for Each Exercise

Now we run the calculations for each of the 5 exercises.


In [ ]:
results = {}

for exercise in df['exercise'].unique():
    exercise_df = df[df['exercise'] == exercise]
    results[exercise] = calculate_metrics(exercise_df)

## Step 6: Display Results Per Exercise

Here are the detailed results for each exercise. Look at the confusion matrix values (TP, FP, TN, FN) to understand where the AI makes mistakes.


In [ ]:
results_df = pd.DataFrame({
    exercise: {
        'Accuracy': f"{metrics['accuracy']:.1%}",
        'Precision': f"{metrics['precision']:.1%}",
        'Recall': f"{metrics['recall']:.1%}",
        'Specificity': f"{metrics['specificity']:.1%}",
        'FPR': f"{metrics['fpr']:.1%}",
        'F1-Score': f"{metrics['f1_score']:.1%}",
        'TP': metrics['TP'],
        'FP': metrics['FP'],
        'TN': metrics['TN'],
        'FN': metrics['FN']
    }
    for exercise, metrics in results.items()
}).T

results_df

,Accuracy,Precision,Recall,Specificity,FPR,F1-Score,TP,FP,TN,FN
bicep_curl,85.0%,81.8%,90.0%,80.0%,20.0%,85.7%,9,2,8,1
lateral_raise,85.0%,88.9%,80.0%,90.0%,10.0%,84.2%,8,1,9,2
overhead_press,90.5%,100.0%,80.0%,100.0%,0.0%,88.9%,8,0,11,2
front_kicks,80.0%,72.7%,88.9%,72.7%,27.3%,80.0%,8,3,8,1
squat,90.0%,90.0%,90.0%,90.0%,10.0%,90.0%,9,1,9,1


## Step 7: Calculate Overall Performance


In [ ]:
num_exercises = len(results)

overall = {
    'accuracy': sum(results[ex]['accuracy'] for ex in results) / num_exercises,
    'precision': sum(results[ex]['precision'] for ex in results) / num_exercises,
    'recall': sum(results[ex]['recall'] for ex in results) / num_exercises,
    'specificity': sum(results[ex]['specificity'] for ex in results) / num_exercises,
    'fpr': sum(results[ex]['fpr'] for ex in results) / num_exercises
}

print("Overall Results (Average Across All Exercises)")
print(f"Overall Accuracy:    {overall['accuracy']:.1%}")
print(f"Overall Precision:   {overall['precision']:.1%}")
print(f"Overall Recall:      {overall['recall']:.1%}")
print(f"Overall Specificity: {overall['specificity']:.1%}")
print(f"Overall FPR:         {overall['fpr']:.1%}")

Overall Results (Average Across All Exercises)
Overall Accuracy:    86.1%
Overall Precision:   86.7%
Overall Recall:      85.8%
Overall Specificity: 86.5%
Overall FPR:         13.5%


## 📊 Results Summary & Assessment

### Overall Performance: **GOOD** ✅

With an **86.1% overall accuracy**, the AI coach is performing well.

- **Squat (90.0%)** and **Overhead Press (90.5%)**: Excellent detection!
- **Bicep Curl (85.0%)** and **Lateral Raise (85.0%)**: Very good performance
- **Precision is 86.7%**: When the AI says you're doing it right, it's almost always correct
- **Recall is 85.8%**: The AI catches most correct forms
- **Low False Positive Rate (13.5%)**: The AI rarely tells you you're correct when you're not

### Areas to Improve 💡
- **Front Kicks (80.0%)**: The lowest accuracy among all exercises
  - Has 3 false positives (said "correct" when it was actually incorrect)
  - This is the trickiest exercise for the AI to evaluate
- **Overhead Press**: Has 0 false positives (100% precision!) but 2 false negatives
  - Sometimes the AI is too strict and says correct form is incorrect